In [2]:
from Utilities import *

import re
import json
import threading
import pandas as pd
import time

/home/bb-visionary/.pyenv/versions/3.8.3/lib/python3.8/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
base_url = 'https://codeforces.com/'
url = 'https://codeforces.com/problemset'
page_url = 'https://codeforces.com/problemset/page/%d'

get the number of pages

In [9]:
soup = openURL(url)
pages = int(soup.find('div', class_='pagination').find_all('li')[-2].text.strip())
pages

62

go to each page and extract all the problems

In [10]:
problems = {
    'ID': [],
    'Name': [],
    'Tags': [],
    'Difficulty': [],
    'Number Solved': [],
    'Page URL': []
}

for page in range(1, pages + 1):
    print(page_url % page, end='')
    added = 0
    discarded = 0
    soup = openURL(page_url % page)
    for tr in soup.find('table', class_='problems').find_all('tr')[1:]:
        tds = tr.find_all('td')
        prob_id = tds[0].text.strip()
        
        td1_divs = tds[1].find_all('div')
        name = td1_divs[0].text.strip()
        link = base_url + td1_divs[0].find('a').get('href')
        tags = '|'.join([tag.text for tag in td1_divs[1].find_all('a')]) 
        
        difficulty = tds[3].text.strip()
        try:
            num_solved = tds[4].text.strip().split('x')[1]
        except:
            num_solved = tds[4].text.strip()
        if(link not in problems['Page URL'] and link != '' and link != None and link != 'nan'):
            problems['ID'].append(prob_id)
            problems['Name'].append(name)
            problems['Tags'].append(tags)
            problems['Difficulty'].append(difficulty)
            problems['Number Solved'].append(num_solved)
            problems['Page URL'].append(link)
            added += 1
        else:
            discarded += 1
#         print('\t%s | %s | %s | %s | %s | %s' % (prob_id, name, categories, difficulty, num_solved, link))
    print(' -> Added: %d | Discarded: %d' % (added, discarded))

https://codeforces.com/problemset/page/1 -> Added: 100 | Discarded: 0
https://codeforces.com/problemset/page/2 -> Added: 100 | Discarded: 0
https://codeforces.com/problemset/page/3 -> Added: 100 | Discarded: 0
https://codeforces.com/problemset/page/4 -> Added: 100 | Discarded: 0
https://codeforces.com/problemset/page/5 -> Added: 100 | Discarded: 0
https://codeforces.com/problemset/page/6 -> Added: 100 | Discarded: 0
https://codeforces.com/problemset/page/7 -> Added: 100 | Discarded: 0
https://codeforces.com/problemset/page/8 -> Added: 100 | Discarded: 0
https://codeforces.com/problemset/page/9 -> Added: 100 | Discarded: 0
https://codeforces.com/problemset/page/10 -> Added: 100 | Discarded: 0
https://codeforces.com/problemset/page/11 -> Added: 100 | Discarded: 0
https://codeforces.com/problemset/page/12 -> Added: 100 | Discarded: 0
https://codeforces.com/problemset/page/13 -> Added: 100 | Discarded: 0
https://codeforces.com/problemset/page/14 -> Added: 100 | Discarded: 0
https://codefor

In [11]:
df = pd.DataFrame(problems)
df.head()

,ID,Name,Tags,Difficulty,Number Solved,Page URL
0,1375I,Cubic Lattice,geometry|math|matrices|number theory,3500,22,https://codeforces.com//problemset/problem/1375/I
1,1375H,Set Merging,constructive algorithms|divide and conquer,3300,62,https://codeforces.com//problemset/problem/1375/H
2,1375G,Tree Modification,brute force|constructive algorithms|dfs and si...,2800,699,https://codeforces.com//problemset/problem/1375/G
3,1375F,Integer Game,constructive algorithms|games|interactive|math,2600,999,https://codeforces.com//problemset/problem/1375/F
4,1375E,Inversion SwapSort,constructive algorithms|greedy|sortings,2500,1346,https://codeforces.com//problemset/problem/1375/E


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6190 entries, 0 to 6189
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             6190 non-null   object
 1   Name           6190 non-null   object
 2   Tags           6190 non-null   object
 3   Difficulty     6190 non-null   object
 4   Number Solved  6190 non-null   object
 5   Page URL       6190 non-null   object
dtypes: object(6)
memory usage: 290.3+ KB


In [13]:
df.to_json('cf_temp.json')

In [ ]:
df.to_html('cf_temp.html', index=False)

In [14]:
df = pd.read_json('cf_temp.json')
df.head()

,ID,Name,Tags,Difficulty,Number Solved,Page URL
0,1375I,Cubic Lattice,geometry|math|matrices|number theory,3500,22,https://codeforces.com//problemset/problem/1375/I
1,1375H,Set Merging,constructive algorithms|divide and conquer,3300,62,https://codeforces.com//problemset/problem/1375/H
2,1375G,Tree Modification,brute force|constructive algorithms|dfs and si...,2800,699,https://codeforces.com//problemset/problem/1375/G
3,1375F,Integer Game,constructive algorithms|games|interactive|math,2600,999,https://codeforces.com//problemset/problem/1375/F
4,1375E,Inversion SwapSort,constructive algorithms|greedy|sortings,2500,1346,https://codeforces.com//problemset/problem/1375/E


In [15]:
items_data = {
    'Completed': [],
    'ID': [],
    'Name': [],
    'Tags': [],
    'Difficulty': [],
    'Number Solved': [],
    'Page URL': [],
    'Time Limit': [],
    'Memory Limit': []
}

def get_item_data(index, df):
    t0 = time.perf_counter()
    soup = openURL(df.iloc[-1])
    try:
        time_limit = soup.find('div', class_='time-limit').contents[-1].split(' ')[0]
    except:
        time_limit = ''
    
    try:
        mem_limit = re.sub('мегабайт', 'megabytes', soup.find('div', class_='memory-limit').contents[-1].strip())
    except:
        mem_limit = ''
    
#     print('\t', time_limit, '|', mem_limit)
    
    items_data['Completed'].append(0)
    items_data['ID'].append(df.iloc[0])
    items_data['Name'].append(df.iloc[1])
    items_data['Tags'].append(df.iloc[2])
    items_data['Difficulty'].append(df.iloc[3])
    items_data['Number Solved'].append(df.iloc[4])
    items_data['Page URL'].append(df.iloc[5])
    items_data['Time Limit'].append(time_limit)
    items_data['Memory Limit'].append(mem_limit)
     
    t1 = time.perf_counter()
    print('\t[%04d/%04d] Successfully scraped [ %s ] [ %s ] { %.2f s }.' % (index, len(items_row), df.iloc[0], df.iloc[3], t1 - t0))

In [16]:
items_row = [row for i, row in df.iterrows()]
len(items_row)

6190

In [17]:
n_threads = 10
n = 0
while(n < len(items_row)):
    t0 = time.perf_counter()
    threads = []
    
    for _ in range(n_threads):
        if(n < len(items_row)):
            thread = threading.Thread(target=get_item_data, args=(n, items_row[n],))
            threads.append(thread)
            thread.start()
            n += 1
    
    for t in threads:
        t.join()
    
    t1 = time.perf_counter()
    print('[%04d/%04d] It took %.2f s' % (n, len(items_row), t1 - t0))

	[0005/6190] Successfully scraped [ 1375D ] [ 1900 ] { 1.35 s }.
	[0007/6190] Successfully scraped [ 1375B ] [ 1200 ] { 1.37 s }.
	[0004/6190] Successfully scraped [ 1375E ] [ 2500 ] { 1.40 s }.
	[0009/6190] Successfully scraped [ 1374F ] [ 2400 ] { 2.91 s }.
	[0002/6190] Successfully scraped [ 1375G ] [ 2800 ] { 2.92 s }.
	[0006/6190] Successfully scraped [ 1375C ] [ 1400 ] { 2.93 s }.
	[0008/6190] Successfully scraped [ 1375A ] [ 1100 ] { 2.95 s }.
	[0001/6190] Successfully scraped [ 1375H ] [ 3300 ] { 2.97 s }.
	[0003/6190] Successfully scraped [ 1375F ] [ 2600 ] { 2.98 s }.
	[0000/6190] Successfully scraped [ 1375I ] [ 3500 ] { 3.00 s }.
[0010/6190] It took 3.00 s
	[0014/6190] Successfully scraped [ 1374B ] [ 900 ] { 1.41 s }.	[0019/6190] Successfully scraped [ 1373D ] [ 1600 ] { 1.43 s }.

	[0015/6190] Successfully scraped [ 1374A ] [ 800 ] { 1.50 s }.
	[0011/6190] Successfully scraped [ 1374E1 ] [ 1600 ] { 1.51 s }.
	[0017/6190] Successfully scraped [ 1373F ] [ 2400 ] { 1.52 s }.

	[0138/6190] Successfully scraped [ 1356A3 ] [  ] { 1.28 s }.	[0139/6190] Successfully scraped [ 1356A2 ] [  ] { 1.28 s }.

	[0137/6190] Successfully scraped [ 1356A4 ] [  ] { 1.41 s }.
	[0135/6190] Successfully scraped [ 1356B1 ] [  ] { 1.45 s }.
	[0130/6190] Successfully scraped [ 1357A1 ] [  ] { 1.45 s }.
	[0133/6190] Successfully scraped [ 1356C ] [  ] { 1.48 s }.	[0134/6190] Successfully scraped [ 1356B2 ] [  ] { 1.51 s }.
	[0132/6190] Successfully scraped [ 1356D1 ] [  ] { 1.52 s }.

	[0136/6190] Successfully scraped [ 1356A5 ] [  ] { 1.55 s }.
	[0131/6190] Successfully scraped [ 1356D2 ] [  ] { 1.57 s }.
[0140/6190] It took 1.57 s
	[0147/6190] Successfully scraped [ 1354G ] [ 2600 ] { 1.21 s }.
	[0140/6190] Successfully scraped [ 1356A1 ] [  ] { 1.23 s }.
	[0142/6190] Successfully scraped [ 1355E ] [ 2100 ] { 1.26 s }.
	[0149/6190] Successfully scraped [ 1354E ] [ 2100 ] { 1.31 s }.
	[0144/6190] Successfully scraped [ 1355C ] [ 1800 ] { 1.35 s }.
	[0148/6190] Successfully scrape

	[0266/6190] Successfully scraped [ 1331H ] [  ] { 1.34 s }.
	[0268/6190] Successfully scraped [ 1331F ] [  ] { 1.36 s }.
	[0264/6190] Successfully scraped [ 1332B ] [ 1400 ] { 2.02 s }.
	[0269/6190] Successfully scraped [ 1331E ] [  ] { 2.41 s }.
	[0267/6190] Successfully scraped [ 1331G ] [  ] { 2.46 s }.
	[0261/6190] Successfully scraped [ 1332E ] [ 2100 ] { 2.55 s }.
	[0265/6190] Successfully scraped [ 1332A ] [ 1100 ] { 2.55 s }.
	[0263/6190] Successfully scraped [ 1332C ] [ 1500 ] { 2.57 s }.
	[0262/6190] Successfully scraped [ 1332D ] [ 1700 ] { 2.58 s }.
	[0260/6190] Successfully scraped [ 1332F ] [ 2500 ] { 2.60 s }.
[0270/6190] It took 2.60 s
	[0279/6190] Successfully scraped [ 1329B ] [ 1700 ] { 1.24 s }.
	[0275/6190] Successfully scraped [ 1330A ] [ 900 ] { 1.26 s }.
	[0278/6190] Successfully scraped [ 1329C ] [ 2400 ] { 1.27 s }.
	[0273/6190] Successfully scraped [ 1331A ] [  ] { 2.53 s }.
	[0271/6190] Successfully scraped [ 1331C ] [  ] { 2.55 s }.
	[0272/6190] Successful

	[0399/6190] Successfully scraped [ 1302E ] [  ] { 1.28 s }.
	[0398/6190] Successfully scraped [ 1302F ] [  ] { 1.30 s }.
	[0395/6190] Successfully scraped [ 1302I ] [  ] { 1.69 s }.
	[0396/6190] Successfully scraped [ 1302H ] [  ] { 2.48 s }.
	[0392/6190] Successfully scraped [ 1303B ] [ 1400 ] { 2.52 s }.
	[0397/6190] Successfully scraped [ 1302G ] [  ] { 2.56 s }.
	[0394/6190] Successfully scraped [ 1302J ] [  ] { 2.58 s }.
	[0393/6190] Successfully scraped [ 1303A ] [ 800 ] { 2.59 s }.
	[0390/6190] Successfully scraped [ 1303D ] [ 1900 ] { 2.65 s }.
	[0391/6190] Successfully scraped [ 1303C ] [ 1600 ] { 2.65 s }.
[0400/6190] It took 2.66 s
	[0402/6190] Successfully scraped [ 1302B ] [  ] { 1.20 s }.
	[0400/6190] Successfully scraped [ 1302D ] [  ] { 1.21 s }.
	[0404/6190] Successfully scraped [ 1301F ] [ 2600 ] { 1.29 s }.
	[0405/6190] Successfully scraped [ 1301E ] [ 2500 ] { 1.33 s }.
	[0401/6190] Successfully scraped [ 1302C ] [  ] { 1.38 s }.
	[0406/6190] Successfully scraped [

	[0513/6190] Successfully scraped [ 1279C ] [ 1400 ] { 2.48 s }.
[0520/6190] It took 2.49 s
	[0521/6190] Successfully scraped [ 1278A ] [ 1000 ] { 1.44 s }.
	[0528/6190] Successfully scraped [ 1276C ] [ 2300 ] { 1.43 s }.
	[0526/6190] Successfully scraped [ 1276E ] [ 3500 ] { 2.36 s }.
	[0525/6190] Successfully scraped [ 1276F ] [ 3400 ] { 2.39 s }.
	[0527/6190] Successfully scraped [ 1276D ] [ 2900 ] { 2.40 s }.
	[0522/6190] Successfully scraped [ 1277D ] [ 1900 ] { 2.44 s }.	[0524/6190] Successfully scraped [ 1277A ] [ 1000 ] { 2.45 s }.

	[0523/6190] Successfully scraped [ 1277B ] [ 1200 ] { 2.49 s }.
	[0529/6190] Successfully scraped [ 1276B ] [ 1900 ] { 2.49 s }.
	[0520/6190] Successfully scraped [ 1278B ] [ 1500 ] { 2.52 s }.
[0530/6190] It took 2.52 s
	[0534/6190] Successfully scraped [ 1275E1 ] [  ] { 1.23 s }.
	[0530/6190] Successfully scraped [ 1276A ] [ 1400 ] { 1.24 s }.
	[0537/6190] Successfully scraped [ 1275B ] [  ] { 1.27 s }.
	[0535/6190] Successfully scraped [ 1275D ]

...

	[5638/6190] Successfully scraped [ 113C ] [ 2200 ] { 0.93 s }.
	[5632/6190] Successfully scraped [ 115B ] [ 1500 ] { 0.96 s }.
	[5631/6190] Successfully scraped [ 115C ] [ 2200 ] { 0.96 s }.
	[5633/6190] Successfully scraped [ 115A ] [ 900 ] { 0.96 s }.
	[5639/6190] Successfully scraped [ 113B ] [ 2000 ] { 0.95 s }.
	[5636/6190] Successfully scraped [ 113E ] [ 2700 ] { 0.97 s }.
	[5637/6190] Successfully scraped [ 113D ] [ 2700 ] { 0.98 s }.
	[5630/6190] Successfully scraped [ 115D ] [ 2600 ] { 1.02 s }.
	[5634/6190] Successfully scraped [ 114B ] [ 1500 ] { 1.01 s }.
	[5635/6190] Successfully scraped [ 114A ] [ 1000 ] { 1.01 s }.
[5640/6190] It took 1.04 s
	[5643/6190] Successfully scraped [ 111E ] [ 2900 ] { 0.95 s }.	[5642/6190] Successfully scraped [ 112A ] [ 800 ] { 0.95 s }.

	[5641/6190] Successfully scraped [ 112B ] [ 1200 ] { 0.96 s }.
	[5644/6190] Successfully scraped [ 111D ] [ 2300 ] { 0.96 s }.
	[5649/6190] Successfully scraped [ 110A ] [ 800 ] { 0.95 s }.
	[5640/6190] Suc

	[5763/6190] Successfully scraped [ 82A ] [ 1100 ] { 0.93 s }.
	[5765/6190] Successfully scraped [ 81D ] [ 2100 ] { 0.93 s }.
	[5769/6190] Successfully scraped [ 80B ] [ 1200 ] { 0.92 s }.
	[5768/6190] Successfully scraped [ 81A ] [ 1400 ] { 0.94 s }.
	[5760/6190] Successfully scraped [ 82D ] [ 2000 ] { 0.97 s }.
	[5762/6190] Successfully scraped [ 82B ] [ 1700 ] { 0.97 s }.
	[5761/6190] Successfully scraped [ 82C ] [ 2000 ] { 0.98 s }.
	[5766/6190] Successfully scraped [ 81C ] [ 1700 ] { 0.97 s }.
	[5767/6190] Successfully scraped [ 81B ] [ 1700 ] { 0.98 s }.
	[5764/6190] Successfully scraped [ 81E ] [ 2700 ] { 1.01 s }.
[5770/6190] It took 1.03 s
	[5772/6190] Successfully scraped [ 79D ] [ 2800 ] { 0.94 s }.
	[5775/6190] Successfully scraped [ 79A ] [ 1200 ] { 0.93 s }.
	[5776/6190] Successfully scraped [ 78E ] [ 2300 ] { 0.92 s }.
	[5773/6190] Successfully scraped [ 79C ] [ 1800 ] { 0.94 s }.
	[5778/6190] Successfully scraped [ 78C ] [ 2000 ] { 0.92 s }.
	[5770/6190] Successfully sc

	[5891/6190] Successfully scraped [ 57E ] [ 3000 ] { 0.98 s }.
	[5890/6190] Successfully scraped [ 58A ] [ 1000 ] { 0.99 s }.
	[5896/6190] Successfully scraped [ 56E ] [ 2200 ] { 0.98 s }.
	[5897/6190] Successfully scraped [ 56D ] [ 2100 ] { 0.98 s }.
	[5898/6190] Successfully scraped [ 56C ] [ 1700 ] { 0.99 s }.
	[5892/6190] Successfully scraped [ 57D ] [ 2500 ] { 1.03 s }.
	[5899/6190] Successfully scraped [ 56B ] [ 1300 ] { 1.02 s }.
	[5893/6190] Successfully scraped [ 57C ] [ 1900 ] { 1.05 s }.
	[5895/6190] Successfully scraped [ 57A ] [ 1300 ] { 1.06 s }.
	[5894/6190] Successfully scraped [ 57B ] [ 1600 ] { 1.08 s }.
[5900/6190] It took 1.09 s
	[5908/6190] Successfully scraped [ 54C ] [ 2000 ] { 0.92 s }.
	[5901/6190] Successfully scraped [ 55E ] [ 2500 ] { 0.95 s }.
	[5900/6190] Successfully scraped [ 56A ] [ 1000 ] { 0.95 s }.
	[5907/6190] Successfully scraped [ 54D ] [ 2100 ] { 0.94 s }.
	[5909/6190] Successfully scraped [ 54B ] [ 1800 ] { 0.94 s }.
	[5902/6190] Successfully sc

	[6021/6190] Successfully scraped [ 36C ] [ 2200 ] { 0.94 s }.
	[6025/6190] Successfully scraped [ 35D ] [ 1700 ] { 0.92 s }.
	[6022/6190] Successfully scraped [ 36B ] [ 1600 ] { 0.94 s }.
	[6024/6190] Successfully scraped [ 35E ] [ 2100 ] { 0.93 s }.
	[6023/6190] Successfully scraped [ 36A ] [ 1300 ] { 0.94 s }.
	[6027/6190] Successfully scraped [ 35B ] [ 1700 ] { 0.94 s }.
	[6028/6190] Successfully scraped [ 35A ] [ 1000 ] { 0.95 s }.
	[6029/6190] Successfully scraped [ 34E ] [ 2000 ] { 0.96 s }.
	[6020/6190] Successfully scraped [ 36D ] [ 2300 ] { 1.00 s }.
	[6026/6190] Successfully scraped [ 35C ] [ 1500 ] { 0.99 s }.
[6030/6190] It took 1.02 s
	[6032/6190] Successfully scraped [ 34B ] [ 900 ] { 0.95 s }.
	[6035/6190] Successfully scraped [ 33D ] [ 2000 ] { 0.95 s }.
	[6030/6190] Successfully scraped [ 34D ] [ 1600 ] { 0.96 s }.
	[6039/6190] Successfully scraped [ 32E ] [ 2400 ] { 0.95 s }.
	[6031/6190] Successfully scraped [ 34C ] [ 1300 ] { 0.97 s }.
	[6037/6190] Successfully scr

	[6156/6190] Successfully scraped [ 8E ] [ 2600 ] { 0.94 s }.	[6159/6190] Successfully scraped [ 8B ] [ 1400 ] { 0.93 s }.

	[6152/6190] Successfully scraped [ 9D ] [ 1900 ] { 0.95 s }.
	[6151/6190] Successfully scraped [ 9E ] [ 2300 ] { 0.95 s }.
	[6153/6190] Successfully scraped [ 9C ] [ 1200 ] { 0.96 s }.
	[6158/6190] Successfully scraped [ 8C ] [ 2000 ] { 0.96 s }.
	[6150/6190] Successfully scraped [ 10A ] [ 900 ] { 0.99 s }.
	[6157/6190] Successfully scraped [ 8D ] [ 2600 ] { 0.98 s }.
	[6155/6190] Successfully scraped [ 9A ] [ 800 ] { 1.00 s }.
	[6154/6190] Successfully scraped [ 9B ] [ 1200 ] { 1.02 s }.
[6160/6190] It took 1.02 s
	[6160/6190] Successfully scraped [ 8A ] [ 1200 ] { 0.96 s }.
	[6166/6190] Successfully scraped [ 6E ] [ 1900 ] { 0.96 s }.
	[6164/6190] Successfully scraped [ 7B ] [ 1600 ] { 0.96 s }.
	[6169/6190] Successfully scraped [ 6B ] [ 1100 ] { 0.96 s }.
	[6165/6190] Successfully scraped [ 7A ] [ 1100 ] { 0.97 s }.
	[6167/6190] Successfully scraped [ 6D ] [ 2

In [18]:
items_df = pd.DataFrame(items_data)
items_df.head()

,Completed,ID,Name,Tags,Difficulty,Number Solved,Page URL,Time Limit,Memory Limit
0,0,1375D,Replace by MEX,brute force|constructive algorithms|sortings,1900,5035,https://codeforces.com//problemset/problem/1375/D,1,256 megabytes
1,0,1375B,Neighbor Grid,constructive algorithms|greedy,1200,10501,https://codeforces.com//problemset/problem/1375/B,1,256 megabytes
2,0,1375E,Inversion SwapSort,constructive algorithms|greedy|sortings,2500,1346,https://codeforces.com//problemset/problem/1375/E,2,256 megabytes
3,0,1374F,Cyclic Shifts Sorting,brute force|constructive algorithms|implementa...,2400,738,https://codeforces.com//problemset/problem/1374/F,2,256 megabytes
4,0,1375G,Tree Modification,brute force|constructive algorithms|dfs and si...,2800,699,https://codeforces.com//problemset/problem/1375/G,1,256 megabytes


In [19]:
items_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6190 entries, 0 to 6189
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Completed      6190 non-null   int64 
 1   ID             6190 non-null   object
 2   Name           6190 non-null   object
 3   Tags           6190 non-null   object
 4   Difficulty     6190 non-null   object
 5   Number Solved  6190 non-null   object
 6   Page URL       6190 non-null   object
 7   Time Limit     6190 non-null   object
 8   Memory Limit   6190 non-null   object
dtypes: int64(1), object(8)
memory usage: 435.4+ KB


In [20]:
items_df['Memory Limit'] = items_df['Memory Limit'].apply(lambda x: re.sub('мегабайт', 'megabytes', x))

In [21]:
items_df['Memory Limit'].value_counts()

256 megabytes     3136
                  2539
512 megabytes      403
64 megabytes        44
1024 megabytes      44
768 megabytes       10
128 megabytes        8
16 megabytes         3
28 megabytes         1
255 megabytes        1
256 mebibytes        1
Name: Memory Limit, dtype: int64

In [22]:
items_df['Time Limit'].value_counts()

        2539
2       1447
1       1329
3        355
4        164
5         96
1.5       47
2.5       42
6         41
0.5       30
7         28
8         22
3.5       18
15        10
10         9
4.5        4
9          2
2.2        1
1.25       1
6.5        1
12         1
7.5        1
1.7        1
5.5        1
Name: Time Limit, dtype: int64

In [23]:
items_df['Difficulty'].value_counts()

1600    348
800     330
2000    320
1900    318
1700    313
1800    309
1500    307
2100    283
2200    281
2400    280
1300    276
1200    265
1400    264
2300    244
1000    236
2500    231
1100    226
2600    187
900     183
2700    172
        149
2900    127
2800    120
3000    102
3100     92
3200     87
3300     52
3400     46
3500     42
Name: Difficulty, dtype: int64

In [14]:
items_df.to_json('codeforces_problems.json', orient='table', index=False)
items_df.head()

,ID,Name,Tags,Difficulty,Number Solved,Page URL,Time Limit,Memory Limit
0,1375D,Replace by MEX,brute force|constructive algorithms|sortings,1900,5035,https://codeforces.com//problemset/problem/1375/D,1,256 megabytes
1,1375B,Neighbor Grid,constructive algorithms|greedy,1200,10501,https://codeforces.com//problemset/problem/1375/B,1,256 megabytes
2,1375E,Inversion SwapSort,constructive algorithms|greedy|sortings,2500,1346,https://codeforces.com//problemset/problem/1375/E,2,256 megabytes
3,1374F,Cyclic Shifts Sorting,brute force|constructive algorithms|implementa...,2400,738,https://codeforces.com//problemset/problem/1374/F,2,256 megabytes
4,1375G,Tree Modification,brute force|constructive algorithms|dfs and si...,2800,699,https://codeforces.com//problemset/problem/1375/G,1,256 megabytes


In [11]:
items_df = pd.read_json('codeforces_problems.json', orient='table')
items_df.head()

,Completed,ID,Name,Tags,Difficulty,Number Solved,Page URL,Time Limit,Memory Limit
0,0,1375D,Replace by MEX,brute force|constructive algorithms|sortings,1900,5035,https://codeforces.com//problemset/problem/1375/D,1,256 megabytes
1,0,1375B,Neighbor Grid,constructive algorithms|greedy,1200,10501,https://codeforces.com//problemset/problem/1375/B,1,256 megabytes
2,0,1375E,Inversion SwapSort,constructive algorithms|greedy|sortings,2500,1346,https://codeforces.com//problemset/problem/1375/E,2,256 megabytes
3,0,1374F,Cyclic Shifts Sorting,brute force|constructive algorithms|implementa...,2400,738,https://codeforces.com//problemset/problem/1374/F,2,256 megabytes
4,0,1375G,Tree Modification,brute force|constructive algorithms|dfs and si...,2800,699,https://codeforces.com//problemset/problem/1375/G,1,256 megabytes


In [12]:
items_df = items_df.drop('Completed', axis=1)

In [13]:
items_df.head()

,ID,Name,Tags,Difficulty,Number Solved,Page URL,Time Limit,Memory Limit
0,1375D,Replace by MEX,brute force|constructive algorithms|sortings,1900,5035,https://codeforces.com//problemset/problem/1375/D,1,256 megabytes
1,1375B,Neighbor Grid,constructive algorithms|greedy,1200,10501,https://codeforces.com//problemset/problem/1375/B,1,256 megabytes
2,1375E,Inversion SwapSort,constructive algorithms|greedy|sortings,2500,1346,https://codeforces.com//problemset/problem/1375/E,2,256 megabytes
3,1374F,Cyclic Shifts Sorting,brute force|constructive algorithms|implementa...,2400,738,https://codeforces.com//problemset/problem/1374/F,2,256 megabytes
4,1375G,Tree Modification,brute force|constructive algorithms|dfs and si...,2800,699,https://codeforces.com//problemset/problem/1375/G,1,256 megabytes
